In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

In [8]:
random_df = pd.concat([
    pd.Series(np.random.random(100),name="x"),
    pd.Series(np.random.random(100),name="y"),
],axis=1)
              

In [22]:
from coinbase.wallet.client import Client
client = Client("VY9bjFOx9rvLnuyO","JQIRUqmye5b3uMVtYyNUdYSUtaYuV6DS")

In [19]:
key ="VY9bjFOx9rvLnuyO"
secret = 'JQIRUqmye5b3uMVtYyNUdYSUtaYuV6DS'

In [24]:
import coinbase

In [75]:
currency="ETH"
pd.DataFrame(
        client.get_historic_prices(
            currency_pair =f'{currency}-USD',period='all').prices
    ).assign(
        **{"currency":currency}
    )

,price,time,currency
0,4527.03,2021-11-26T00:00:00Z,ETH
1,3998.57,2021-11-19T00:00:00Z,ETH
2,4721.07,2021-11-12T00:00:00Z,ETH
3,4867.81,2021-11-10T00:00:00Z,ETH
4,4537.14,2021-11-05T00:00:00Z,ETH
...,...,...,...
327,1.33801,2015-09-06T00:00:00Z,ETH
328,1.18299,2015-08-30T00:00:00Z,ETH
329,1.375,2015-08-23T00:00:00Z,ETH
330,1.68435,2015-08-16T00:00:00Z,ETH


In [93]:

CURRENCIES = ["ETH","BTC"]
TIMESTAMP = "Timestamp (flt)"
all_prices = pd.DataFrame()

for currency in CURRENCIES:
    current_prices =pd.DataFrame(
        client.get_historic_prices(
            currency_pair =f'{currency}-USD',period='all').prices
    ).assign(
        **{"currency":currency}
    ).apply(
        lambda x: x.astype(float) if x.name =="price" else 
        ( pd.to_datetime(x) if x.name=="time" else x )
        )
    all_prices = pd.concat([
        all_prices,
        current_prices
    ])


# price_records = pd.DataFrame(
#     prices.prices,
# )
# price_records["time_object"] =  pd.to_datetime(
#     price_records["time"]
# )

all_prices[TIMESTAMP] = all_prices["time"].apply(lambda x: x.timestamp())
DAY = "Day"
all_prices[DAY ] = all_prices[TIMESTAMP] /1000/60/24

In [66]:
# price_records["price"] = price_records["price"].astype(float)

In [94]:
px.scatter(
    all_prices, 
    x= DAY,
    y= "price",
    color="currency",
    title = f'''value of various cryptos in dollars<br><sup>{",".join(CURRENCIES)}''',
)

In [99]:
CURRENCY = "currency"
DAY_INT = "Day int"
PRICE = "price"

all_prices[DAY_INT] = all_prices[DAY].apply(np.round)
daily_prices = all_prices.groupby([DAY_INT,CURRENCY])[PRICE].mean()

In [104]:
daily_prices.reset_index()

,Day int,currency,price
0,955.0,BTC,105.120000
1,956.0,BTC,112.562000
2,957.0,BTC,128.066500
3,958.0,BTC,130.530000
4,959.0,BTC,123.410000
...,...,...,...
317,1135.0,ETH,3922.515000
318,1136.0,BTC,63446.443333
319,1136.0,ETH,4564.400000
320,1137.0,BTC,61307.885000


In [105]:
byday =daily_prices.reset_index()

In [108]:
byday.set_index([CURRENCY,DAY_INT])[PRICE].unstack(CURRENCY)

currency,BTC,ETH
Day int,,
955.0,105.120000,NaN
956.0,112.562000,NaN
957.0,128.066500,NaN
958.0,130.530000,NaN
959.0,123.410000,NaN
...,...,...
1133.0,50225.975000,3496.545000
1134.0,49259.560000,3248.130000
1135.0,60867.050000,3922.515000


In [109]:
px.scatter(
    byday.set_index([CURRENCY,DAY_INT])[PRICE].unstack(CURRENCY),
    x="BTC",
    y="ETH"
)

In [112]:
as_columns = byday.set_index([CURRENCY,DAY_INT])[PRICE].unstack(CURRENCY)

In [114]:
btc_eth_ratio = as_columns["BTC"] / as_columns["ETH"]

In [116]:
btc_eth_ratio

Day int
955.0           NaN
956.0           NaN
957.0           NaN
958.0           NaN
959.0           NaN
            ...    
1133.0    14.364458
1134.0    15.165514
1135.0    15.517353
1136.0    13.900281
1137.0    13.884520
Length: 183, dtype: float64

In [118]:
px.histogram(
    btc_eth_ratio,
#     x="BTC",
#     y="ETH",
    title = "frequency of price ratio values btw BTC and ETH",
    nbins =100,
    marg
)